Starting from the general spherical harmonics moment equations, we will discretize the equations onto staggered grids.

In [12]:
%matplotlib inline
%config InlineBackend.figure_format='retina'
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
import os
#from util import *

In [37]:
def a_lm( l, m ):
    return np.sqrt((l-m+1)*(l+m+1)/((2*l+3)*(2*l+1)))
def b_lm( l, m ):
    return np.sqrt((l-m)*(l+m)/((2*l+1)*(2*l-1)))
def c_lm( l, m ):
    return np.sqrt((l+m+1)*(l+m+2)/((2*l+3)*(2*l+1)))
def d_lm( l, m ):
    return np.sqrt((l-m)*(l-m-1)/((2*l+1)*(2*l-1)))
def e_lm( l, m ):
    return np.sqrt((l-m+1)*(l-m+2)/((2*l+3)*(2*l+1)))
def f_lm( l, m ):
    return np.sqrt((l+m)*(l+m-1)/((2*l+1)*(2*l-1)))

# computes the real part coefficients for a row in the S matrix
def R_lm( l, m, lm_to_index ):
    numCoeffs = len(index_to_lm)
    if m == 0:
        #return L_lm[lm_to_index[(l,m)]]
        coeffs = np.zeros(numCoeffs, dtype=complex)
        coeffs[lm_to_index[(l,m)]] = 1.0
        return coeffs
    a = np.power(-1.0, m)/np.sqrt(2)
    b = np.power(-1.0, 2.0*m)/np.sqrt(2)
    #result = a*L_lm[lm_to_index[(l,m)]] + b*L_lm[lm_to_index[(l,-m)]]
    #print("{} test".format(result))
    coeffs = np.zeros(numCoeffs, dtype=complex)
    coeffs[lm_to_index[(l,m)]] = a
    if (l,-m) in lm_to_index:
        coeffs[lm_to_index[(l,-m)]] = b
    return coeffs

    
# computes the imaginary part coefficients for a row in the S matrix
def I_lm( l, m, lm_to_index ):
    numCoeffs = len(index_to_lm)
    a = np.power(-1.0, m)/np.sqrt(2)*1j
    b = -np.power(-1.0, 2*m)/np.sqrt(2)*1j
    #result = a*L_lm[lm_to_index[(l,m)]] + b*L_lm[lm_to_index[(l,-m)]]
    coeffs = np.zeros(numCoeffs, dtype=complex)
    coeffs[lm_to_index[(l,m)]] = a
    if (l,-m) in lm_to_index:
        coeffs[lm_to_index[(l,-m)]] = b
    return coeffs

We start by building the solution vector. We are solving the general 3d moment equations, but for a problem where the solution is independent of $z$. This causes all moments for which $(l+m)$ is odd to vanish.



In [86]:
N = 3 # truncation order



# this array holds the l,m indices associated with each equation
# NB: we dont use l(l+1)+m because in 2d the sequence changes (this is because we skipp odd (l+m) entries)
index_to_lm = []
lm_to_index = {} # this dict will map l,m indices to the equation index
for l in range(0, N+1):
    for m in range(-l, l+1):
        # in 2d, we only need to solve for moments where l+m is even
        if (l+m) % 2 == 0:
            index_to_lm.append( (l, m) )
            lm_to_index[(l,m)] = len(index_to_lm)-1
            
            
#print(index_to_lm)
#print(lm_to_index)
numCoeffs = len(index_to_lm)
print("numCoeffs={}".format(numCoeffs))

# now given the indices, we can assemble the Mx, My, C and S matrices
Mx_complex = np.zeros((numCoeffs, numCoeffs),dtype=complex)
My_complex = np.zeros((numCoeffs, numCoeffs),dtype=complex)
C = np.zeros((numCoeffs, numCoeffs))
S = np.zeros((numCoeffs, numCoeffs),dtype=complex)



count = 0
# iterate all moment equations
#for (l,m) in lm_to_index:
for i in range(len(index_to_lm)):
    l = index_to_lm[i][0]
    m = index_to_lm[i][1]
    #print("i={} l={} m={}".format(i, l, m))
    #continue;
    #i = lm_to_index[(l,m)]
    #print("l={} m={}".format(l, m))    
    # build Mx and My matrix ---
    if (l-1,m-1) in lm_to_index:
        #pass
        Mx_complex[i, lm_to_index[(l-1,m-1)]] = -0.5*c_lm(l-1, m-1)
        #My_complex[i, lm_to_index[(l-1,m-1)]] = 0.5*1j*c_lm(l-1, m-1)
    if (l+1,m-1) in lm_to_index:
        #pass
        Mx_complex[i, lm_to_index[(l+1,m-1)]] = 0.5*d_lm(l+1, m-1)   
        #My_complex[i, lm_to_index[(l+1,m-1)]] = -0.5*1j*d_lm(l+1, m-1)   
    if (l-1,m+1) in lm_to_index:
        #pass
        Mx_complex[i, lm_to_index[(l-1,m+1)]] = 0.5*e_lm(l-1, m+1)
        #My_complex[i, lm_to_index[(l-1,m+1)]] = 0.5*1j*e_lm(l-1, m+1)
    if (l+1,m+1) in lm_to_index:
        #pass
        Mx_complex[i, lm_to_index[(l+1,m+1)]] = -0.5*f_lm(l+1, m+1)
        #My_complex[i, lm_to_index[(l+1,m+1)]] = -0.5*1j*f_lm(l+1, m+1)


for l in range(0, N+1):
    for m in range(l+1, -1, -1):
        if (l+m) % 2 != 0:
            continue
        # build S matrix, which converts solution from complex to real values ---
        # real part
        S[count] = R_lm(l, m, lm_to_index)
        count+=1

        if m > 0:
            # imaginary part
            S[count] = I_lm(l, m, lm_to_index)
            count+=1
    
    
S_inv = np.linalg.inv(S)
print( np.real(S.dot(Mx_complex.dot(S_inv))) )





numCoeffs=10
[[ 0.          0.57735027  0.          0.          0.          0.          0.
   0.          0.          0.        ]
 [ 0.57735027  0.          0.          0.4472136   0.         -0.25819889
   0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.4472136   0.          0.
   0.          0.          0.        ]
 [ 0.          0.4472136   0.          0.          0.          0.
   0.46291005  0.         -0.11952286  0.        ]
 [ 0.          0.          0.4472136   0.          0.          0.          0.
   0.46291005  0.         -0.11952286]
 [ 0.         -0.25819889  0.          0.          0.          0.          0.
   0.          0.41403934  0.        ]
 [ 0.          0.          0.          0.46291005  0.          0.          0.
   0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.46291005  0.          0.
   0.          0.          0.        ]
 [ 0.          0.          0.         -0.11